### Import Modules

In [1]:
from google import genai
import PIL.Image
from pydantic import BaseModel, Field
from typing import List
import os

### Define Pydantic models

In [2]:
class LineItem(BaseModel):
    description: str
    quantity: int
    unit_price: float
    total: float

class Invoice(BaseModel):
    invoice_number: str
    date: str
    vendor_name: str
    line_items: List[LineItem]
    total_amount: float

### Initialize Client

In [3]:
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])
image = PIL.Image.open('data/invoice.jpg')

### Define Prompt

In [4]:
prompt="Extract the invoice details as structured data"

### Generate Structured Data with Schema Validation

In [5]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[prompt, image],
    config={"response_mime_type": "application/json", "response_schema": Invoice}
)

### Access the validated data

In [6]:
invoice_data = response.parsed

In [7]:
print(type(invoice_data))
print(invoice_data.line_items)
print(f"Invoice #{invoice_data.invoice_number} from {invoice_data.vendor_name}")
print(f"Total amount: ${invoice_data.total_amount}")

<class '__main__.Invoice'>
[LineItem(description='Cappuccino (Tall)', quantity=1, unit_price=210.0, total=210.0), LineItem(description='Hot Chocolate (Tall)', quantity=1, unit_price=230.0, total=230.0), LineItem(description='CGST @ 2.5%', quantity=1, unit_price=11.0, total=11.0), LineItem(description='SGST @ 2.5%', quantity=1, unit_price=11.0, total=11.0)]
Invoice #HYD04250987 from STARBUCKS
Total amount: $462.0
